In [ ]:
import os

from datasets import load_dataset

from transformers import (
    AutoModelForCausalLM,
    AutoModelForSequenceClassification,
    AutoTokenizer,
)

from accelerate import PartialState

from trl import ModelConfig, PPOConfig, PPOTrainer, ScriptArguments
from trl.trainer.utils import SIMPLE_CHAT_TEMPLATE

from utils import PolicyMixture


os.environ["CUDA_VISIBLE_DEVICES"] = "6, 7"

In [3]:
###############################################################################
# Configs
###############################################################################

script_args = ScriptArguments(
    dataset_name = "trl-internal-testing/descriptiveness-sentiment-trl-style",
    dataset_train_split = "descriptiveness",
)

# Model to use for policies
# =============================================================================

model_config = ModelConfig(
    model_name_or_path  = "EleutherAI/pythia-70m-deduped",
    trust_remote_code   = False,
)

# PPO trainers' configs
# =============================================================================

ppo_config_1 = PPOConfig(
    reward_model_path   = "EleutherAI/pythia-70m-deduped",
    dataset_num_proc    = 1,
    output_dir          = "./policy_1",
    num_train_epochs    = 0.01,
    num_ppo_epochs      = 1,
    num_mini_batches    = 1,
    learning_rate       = 3e-6,
    # total_episodes      = 10000,
    missing_eos_penalty = 1.0,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 16,
    local_rollout_forward_batch_size = 1,
    push_to_hub = False,
)

# -----------------------------------------------------------------------------

ppo_config_2 = PPOConfig(
    reward_model_path   = "EleutherAI/pythia-70m-deduped",
    dataset_num_proc    = 1,
    output_dir          = "./policy_2",
    num_train_epochs    = 0.01,
    num_ppo_epochs      = 1,
    num_mini_batches    = 1,
    learning_rate       = 3e-6,
    # total_episodes      = 10000,
    missing_eos_penalty = 1.0,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 16,
    local_rollout_forward_batch_size = 1,
    push_to_hub = False,
)


In [4]:
###############################################################################
#  Tokenizer
###############################################################################

tokenizer = AutoTokenizer.from_pretrained(
    model_config.model_name_or_path,
    padding_side="left",
    trust_remote_code=model_config.trust_remote_code,
)
tokenizer.add_special_tokens({"pad_token": "[PAD]"})

if tokenizer.chat_template is None:
    tokenizer.chat_template = SIMPLE_CHAT_TEMPLATE

In [5]:
###############################################################################
#  Models
###############################################################################

# Policy 1
# -----------------------------------------------------------------------------

policy_1 = AutoModelForCausalLM.from_pretrained(
    model_config.model_name_or_path
)
sft_policy_1 = AutoModelForCausalLM.from_pretrained(
    model_config.model_name_or_path
)
value_model_1 = AutoModelForSequenceClassification.from_pretrained(
    model_config.model_name_or_path, 
    num_labels=1
)
reward_model_1 = AutoModelForSequenceClassification.from_pretrained(
    ppo_config_1.reward_model_path,
    num_labels=1
)

# Policy 2
# -----------------------------------------------------------------------------

policy_2 = AutoModelForCausalLM.from_pretrained(
    model_config.model_name_or_path
)

# Ref policy 1
# -----------------------------------------------------------------------------

ref_policy_1 = PolicyMixture([sft_policy_1, policy_2], [0.8, 0.2])

Some weights of GPTNeoXForSequenceClassification were not initialized from the model checkpoint at EleutherAI/pythia-70m-deduped and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of GPTNeoXForSequenceClassification were not initialized from the model checkpoint at EleutherAI/pythia-70m-deduped and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
###############################################################################
#  Dataset
###############################################################################

dataset = load_dataset(
    script_args.dataset_name,
    split=script_args.dataset_train_split
)
eval_samples = 100
train_dataset = dataset.select(range(len(dataset) - eval_samples))
eval_dataset = dataset.select(range(len(dataset) - eval_samples, len(dataset)))
dataset_text_field = "prompt"

def prepare_dataset(dataset, tokenizer):
    """
    pre-tokenize the dataset before training; only collate during training
    """

    def tokenize(element):
        outputs = tokenizer(
            element[dataset_text_field],
            padding=False,
        )
        return {"input_ids": outputs["input_ids"]}

    return dataset.map(
        tokenize,
        batched=True,
        remove_columns=dataset.column_names
    )

# Compute that only on the main process for faster data processing.
# see: https://github.com/huggingface/trl/pull/1255
with PartialState().local_main_process_first():
    train_dataset = prepare_dataset(train_dataset, tokenizer)
    eval_dataset = prepare_dataset(eval_dataset, tokenizer)


In [7]:
###############################################################################
#  Training
###############################################################################

NUM_EPOCHS = 2

trainer = PPOTrainer(
    config            = ppo_config_1,
    processing_class  = tokenizer,
    policy            = policy_1,
    ref_policy        = ref_policy_1,
    reward_model      = reward_model_1,
    value_model       = value_model_1,
    train_dataset     = train_dataset,
    eval_dataset      = eval_dataset,
)

for epoch in range(NUM_EPOCHS):
    trainer.train()
    ref_policy_1 = PolicyMixture([sft_policy_1, policy_2], coefs=[0.8, 0.2])
    trainer.ref_policy = ref_policy_1

===training policy===


From v4.47 onwards, when a model cache is to be returned, `generate` will return a `Cache` instance instead by default (as opposed to the legacy tuple of tuples format). If you want to keep returning the legacy format, please set `return_legacy_cache=True`.


Step,Training Loss


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ query                                       ┃ model response                               ┃ score              ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│ She couldn't get the saw-player the kid had │                                              │ -3.950528621673584 │
│ mentioned out of her mind. Sounds Hawaiian, │                                              │                    │
│ she thought over and over again as Eddie    │ "I'm sorry, Eddie," Eddie said. "I'm sorry,  │                    │
│ pushed her grimly along in the new          │ Eddie."                                      │                    │
│ wheelchair, weaving in and out of the       │                                              │                    │
│ stalled vehicles. Sounds Hawaiian, doesn't  │ "I'm sorry, Eddie," Eddie said. "I'm sorry,  │                    │
│ it? Sounds fucking Hawaiian, doesn't it.    │ Eddie."                                      │                    │
│                                             │                                              │                    │
│                                             │ "I'm sorry, Eddie," Eddie said. "I'm         │                    │
├─────────────────────────────────────────────┼──────────────────────────────────────────────┼────────────────────┤
│ "You little piss-ant!" the girl snapped.    │  "I don't know what to do. I don't know what │ -4.41873836517334  │
│ "Don't tell me I slipped up. She died at    │ to do. I don't know what to do. I don't know │                    │
│ seventeen. That's why I wasn't there. I was │ what to do. I don't know what to do. I don't │                    │
│ never notified."                            │ know what to do. I don't know                │                    │
│                                             │                                              │                    │
│ "But I don't do sixteen," he said, his      │                                              │                    │
│ voice going nasty.                          │                                              │                    │
├─────────────────────────────────────────────┼──────────────────────────────────────────────┼────────────────────┤
│ Daniel flashed one of his own-a real one,   │  "I'm not sure I'm going to be able to go to │ -2.727306842803955 │
│ this time.                                  │ the hospital."                               │                    │
│                                             │                                              │                    │
│ "I almost had a heart attack when Mom       │ "I'm not sure I'm going to be able to go to  │                    │
│ almost had a heart attack," he said, his    │ the hospital."                               │                    │
│ voice quiet. Serious. "I'm-I'm happy you're │                                              │                    │
│ okay."                                      │ "I'm not sure I'm going to be able to go to  │                    │
│                                             │ the                                          │                    │
│ I looked around the room.                   │                                              │                    │
├─────────────────────────────────────────────┼──────────────────────────────────────────────┼────────────────────┤
│ "Lights," the Oracle announced.  Images     │  I'm not sure what to do."                   │ -4.191888332366943 │
│ sprang up on the panels, showing coral and  │ "I'm not sure what to do."                   │                    │
│ the calm swirling of sea particles.         │ "I'm not sure what to do."                   │                    │
│ "What am I looking at?" asked Zook.         │ "I'm not

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ query                                       ┃ model response                              ┃ score               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ She couldn't get the saw-player the kid had │                                             │ -4.774353981018066  │
│ mentioned out of her mind. Sounds Hawaiian, │                                             │                     │
│ she thought over and over again as Eddie    │ "I'm sorry, Eddie," Eddie said. "I'm sorry, │                     │
│ pushed her grimly along in the new          │ Eddie. I'm sorry, Eddie. I'm sorry, Eddie.  │                     │
│ wheelchair, weaving in and out of the       │ I'm sorry, Eddie. I'm sorry, Eddie. I'm     │                     │
│ stalled vehicles. Sounds Hawaiian, doesn't  │ sorry, Eddie. I'm sorry,                    │                     │
│ it? Sounds fucking Hawaiian, doesn't it.    │                                             │                     │
├─────────────────────────────────────────────┼─────────────────────────────────────────────┼─────────────────────┤
│ "You little piss-ant!" the girl snapped.    │  "I don't know what to do. I don't know     │ -4.41873836517334   │
│ "Don't tell me I slipped up. She died at    │ what to do. I don't know what to do. I      │                     │
│ seventeen. That's why I wasn't there. I was │ don't know what to do. I don't know what to │                     │
│ never notified."                            │ do. I don't know what to do. I don't know   │                     │
│                                             │                                             │                     │
│ "But I don't do sixteen," he said, his      │                                             │                     │
│ voice going nasty.                          │                                             │                     │
├─────────────────────────────────────────────┼─────────────────────────────────────────────┼─────────────────────┤
│ Daniel flashed one of his own-a real one,   │  "I'm sorry, Mom. I'm sorry. I'm sorry. I'm │ -3.3509974479675293 │
│ this time.                                  │ sorry. I'm sorry. I'm sorry. I'm sorry. I'm │                     │
│                                             │ sorry. I'm sorry. I'm sorry. I'm sorry. I'm │                     │
│ "I almost had a heart attack when Mom       │ sorry. I'm                                  │                     │
│ almost had a heart attack," he said, his    │                                             │                     │
│ voice quiet. Serious. "I'm-I'm happy you're │                                             │                     │
│ okay."                                      │                                             │                     │
│                                             │                                             │                     │
│ I looked around the room.                   │                                             │                     │
├─────────────────────────────────────────────┼─────────────────────────────────────────────┼─────────────────────┤
│ "Lights," the Oracle announced.  Images     │  I'm not sure what to do."                  │ -4.191888332366943  │
│ sprang up on the panels, showing coral and  │ "I'm not sure what to do."                  │                     │
│ the calm swirling of sea particles.         │ "I'm not sure what to do."                  │                     │
│ "What am I looking at?" asked Zook.         │ "I'm not sure what to do."                  │                     │
│ "The extra eyes of technology that I        │ "I'm not sure what to do."                  │                     │
│ dropped behind us.                          │ "I'm not

...

Step,Training Loss


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ query                                       ┃ model response                              ┃ score               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ She couldn't get the saw-player the kid had │                                             │ -3.950528621673584  │
│ mentioned out of her mind. Sounds Hawaiian, │                                             │                     │
│ she thought over and over again as Eddie    │ "I'm sorry, Eddie," Eddie said. "I'm sorry, │                     │
│ pushed her grimly along in the new          │ Eddie."                                     │                     │
│ wheelchair, weaving in and out of the       │                                             │                     │
│ stalled vehicles. Sounds Hawaiian, doesn't  │ "I'm sorry, Eddie," Eddie said. "I'm sorry, │                     │
│ it? Sounds fucking Hawaiian, doesn't it.    │ Eddie."                                     │                     │
│                                             │                                             │                     │
│                                             │ "I'm sorry, Eddie," Eddie said. "I'm        │                     │
├─────────────────────────────────────────────┼─────────────────────────────────────────────┼─────────────────────┤
│ "You little piss-ant!" the girl snapped.    │  "I don't know what to do."                 │ -5.0219292640686035 │
│ "Don't tell me I slipped up. She died at    │                                             │                     │
│ seventeen. That's why I wasn't there. I was │ "I don't know what to do."                  │                     │
│ never notified."                            │                                             │                     │
│                                             │ "I don't know what to do."                  │                     │
│ "But I don't do sixteen," he said, his      │                                             │                     │
│ voice going nasty.                          │ "I don't know what to do."                  │                     │
│                                             │                                             │                     │
│                                             │ "I don't know what to do."                  │                     │
├─────────────────────────────────────────────┼─────────────────────────────────────────────┼─────────────────────┤
│ Daniel flashed one of his own-a real one,   │  "I'm not sure I'm going to be able to go   │ -2.727306842803955  │
│ this time.                                  │ to the hospital."                           │                     │
│                                             │                                             │                     │
│ "I almost had a heart attack when Mom       │ "I'm not sure I'm going to be able to go to │                     │
│ almost had a heart attack," he said, his    │ the hospital."                              │                     │
│ voice quiet. Serious. "I'm-I'm happy you're │                                             │                     │
│ okay."                                      │ "I'm not sure I'm going to be able to go to │                     │
│                                             │ the                                         │                     │
│ I looked around the room.                   │                                             │                     │
├─────────────────────────────────────────────┼─────────────────────────────────────────────┼─────────────────────┤
│ "Lights," the Oracle announced.  Images     │  I'm not sure what to do."                  │ -6.4303083419799805 │
│ sprang up on the panels, showing coral and  │         

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ query                                       ┃ model response                              ┃ score               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ She couldn't get the saw-player the kid had │                                             │ -3.950528621673584  │
│ mentioned out of her mind. Sounds Hawaiian, │                                             │                     │
│ she thought over and over again as Eddie    │ "I'm sorry, Eddie," Eddie said. "I'm sorry, │                     │
│ pushed her grimly along in the new          │ Eddie."                                     │                     │
│ wheelchair, weaving in and out of the       │                                             │                     │
│ stalled vehicles. Sounds Hawaiian, doesn't  │ "I'm sorry, Eddie," Eddie said. "I'm sorry, │                     │
│ it? Sounds fucking Hawaiian, doesn't it.    │ Eddie."                                     │                     │
│                                             │                                             │                     │
│                                             │ "I'm sorry, Eddie," Eddie said. "I'm        │                     │
├─────────────────────────────────────────────┼─────────────────────────────────────────────┼─────────────────────┤
│ "You little piss-ant!" the girl snapped.    │  "I don't know what to do."                 │ -5.0219292640686035 │
│ "Don't tell me I slipped up. She died at    │                                             │                     │
│ seventeen. That's why I wasn't there. I was │ "I don't know what to do."                  │                     │
│ never notified."                            │                                             │                     │
│                                             │ "I don't know what to do."                  │                     │
│ "But I don't do sixteen," he said, his      │                                             │                     │
│ voice going nasty.                          │ "I don't know what to do."                  │                     │
│                                             │                                             │                     │
│                                             │ "I don't know what to do."                  │                     │
├─────────────────────────────────────────────┼─────────────────────────────────────────────┼─────────────────────┤
│ Daniel flashed one of his own-a real one,   │  "I'm not sure I can."                      │ -4.304604530334473  │
│ this time.                                  │                                             │                     │
│                                             │ "I'm sure you can."                         │                     │
│ "I almost had a heart attack when Mom       │                                             │                     │
│ almost had a heart attack," he said, his    │ "I'm sure you can."                         │                     │
│ voice quiet. Serious. "I'm-I'm happy you're │                                             │                     │
│ okay."                                      │ "I'm sure you can."                         │                     │
│                                             │                                             │                     │
│ I looked around the room.                   │ "I'm sure you can."                         │                     │
│                                             │                                             │                     │
│                                             │ "I'm sure you can."                         │                     │
├─────────────────────────────────────────────┼─────────

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ query                                       ┃ model response                              ┃ score               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ She couldn't get the saw-player the kid had │                                             │ -3.950528621673584  │
│ mentioned out of her mind. Sounds Hawaiian, │                                             │                     │
│ she thought over and over again as Eddie    │ "I'm sorry, Eddie," Eddie said. "I'm sorry, │                     │
│ pushed her grimly along in the new          │ Eddie."                                     │                     │
│ wheelchair, weaving in and out of the       │                                             │                     │
│ stalled vehicles. Sounds Hawaiian, doesn't  │ "I'm sorry, Eddie," Eddie said. "I'm sorry, │                     │
│ it? Sounds fucking Hawaiian, doesn't it.    │ Eddie."                                     │                     │
│                                             │                                             │                     │
│                                             │ "I'm sorry, Eddie," Eddie said. "I'm        │                     │
├─────────────────────────────────────────────┼─────────────────────────────────────────────┼─────────────────────┤
│ "You little piss-ant!" the girl snapped.    │  "I don't know what to do."                 │ -5.0219292640686035 │
│ "Don't tell me I slipped up. She died at    │                                             │                     │
│ seventeen. That's why I wasn't there. I was │ "I don't know what to do."                  │                     │
│ never notified."                            │                                             │                     │
│                                             │ "I don't know what to do."                  │                     │
│ "But I don't do sixteen," he said, his      │                                             │                     │
│ voice going nasty.                          │ "I don't know what to do."                  │                     │
│                                             │                                             │                     │
│                                             │ "I don't know what to do."                  │                     │
├─────────────────────────────────────────────┼─────────────────────────────────────────────┼─────────────────────┤
│ Daniel flashed one of his own-a real one,   │  "I'm not sure I can."                      │ -3.3905396461486816 │
│ this time.                                  │                                             │                     │
│                                             │ "I'm not sure I can."                       │                     │
│ "I almost had a heart attack when Mom       │                                             │                     │
│ almost had a heart attack," he said, his    │ "I'm not sure I can."                       │                     │
│ voice quiet. Serious. "I'm-I'm happy you're │                                             │                     │
│ okay."                                      │ "I'm not sure I can."                       │                     │
│                                             │                                             │                     │
│ I looked around the room.                   │ "I'm not sure I can."                       │                     │
│                                             │                                             │                     │
│                                             │ "I'm                                        │                     │
├─────────────────────────────────────────────┼─────────

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ query                                       ┃ model response                              ┃ score               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ She couldn't get the saw-player the kid had │                                             │ -3.950528621673584  │
│ mentioned out of her mind. Sounds Hawaiian, │                                             │                     │
│ she thought over and over again as Eddie    │ "I'm sorry, Eddie," Eddie said. "I'm sorry, │                     │
│ pushed her grimly along in the new          │ Eddie."                                     │                     │
│ wheelchair, weaving in and out of the       │                                             │                     │
│ stalled vehicles. Sounds Hawaiian, doesn't  │ "I'm sorry, Eddie," Eddie said. "I'm sorry, │                     │
│ it? Sounds fucking Hawaiian, doesn't it.    │ Eddie."                                     │                     │
│                                             │                                             │                     │
│                                             │ "I'm sorry, Eddie," Eddie said. "I'm        │                     │
├─────────────────────────────────────────────┼─────────────────────────────────────────────┼─────────────────────┤
│ "You little piss-ant!" the girl snapped.    │  "I don't know what to do."                 │ -5.0219292640686035 │
│ "Don't tell me I slipped up. She died at    │                                             │                     │
│ seventeen. That's why I wasn't there. I was │ "I don't know what to do."                  │                     │
│ never notified."                            │                                             │                     │
│                                             │ "I don't know what to do."                  │                     │
│ "But I don't do sixteen," he said, his      │                                             │                     │
│ voice going nasty.                          │ "I don't know what to do."                  │                     │
│                                             │                                             │                     │
│                                             │ "I don't know what to do."                  │                     │
├─────────────────────────────────────────────┼─────────────────────────────────────────────┼─────────────────────┤
│ Daniel flashed one of his own-a real one,   │  "I'm sorry, Mom. I'm sorry."               │ -4.522566795349121  │
│ this time.                                  │                                             │                     │
│                                             │ "I'm sorry, Mom. I'm sorry. I'm sorry. I'm  │                     │
│ "I almost had a heart attack when Mom       │ sorry. I'm sorry. I'm sorry. I'm sorry. I'm │                     │
│ almost had a heart attack," he said, his    │ sorry. I'm sorry. I                         │                     │
│ voice quiet. Serious. "I'm-I'm happy you're │                                             │                     │
│ okay."                                      │                                             │                     │
│                                             │                                             │                     │
│ I looked around the room.                   │                                             │                     │
├─────────────────────────────────────────────┼─────────────────────────────────────────────┼─────────────────────┤
│ "Lights," the Oracle announced.  Images     │  I'm not sure what to do."                  │ -6.4303083419799805 │
│ sprang up on the panels, showing coral and  │         